In [ ]:
import numpy as np
import seaborn as sb 
from TweetHandler import *
#from cities_loc import cities_loc
from Translator import GoogleTranslate
import pandas as pd
import time
import math 
#from googletrans import Translator 
#translator = Translator() 

import torch
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import apex

from unidecode import unidecode
from nltk.tokenize import TweetTokenizer
#from stop_words import get_stop_words
import nltk
tknzr = TweetTokenizer()
from nltk.corpus import stopwords
stop = stopwords.words('english')   
import re
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)    


In [ ]:
cities_locations  = cities_l

### Download Tweets from the specified cities


In [ ]:
def ingest():
    data = pd.read_csv("all_tweets.csv") # Enter your file location
    #data.drop(['username','date','retweets','favorites'], axis=1, inplace=True)
    data.drop_duplicates(inplace=True)
    data = data[data['Tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('Dataset loaded with shape', data.shape  )  
    return data

data = ingest()

In [ ]:
data.head()

In [ ]:
data_copy = data.copy() 
stop += [ 'KOT','kot', '<elong>','com','pic.twitter.com','twitter','rt','que','pic.twitter.com/dnfh8aa3vz','lol']

data_copy['Tweet'] = data_copy['Tweet'].map(lambda x:re.sub('[^a-zA-Z]',' ',str(x)))
#remove links or anything starting with http
data_copy['Tweet'] = data_copy['Tweet'].map(lambda x:re.sub('http.*','',str(x)))
#lower case everything 
data_copy['Tweet'] = data_copy['Tweet'].map(lambda x:re.sub(r'#','',str(x)))
data_copy['Tweet'] = data_copy['Tweet'].map(lambda x:re.sub(r'@\w*','',str(x)))
data_copy['Tweet'] = data_copy['Tweet'].map(lambda x:str(x).lower()) 
data_copy['Tweet'] = data_copy['Tweet'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) > 3))
data_copy['Tweet'] = data_copy['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
data_copy.head(10)

In [ ]:
#clean_tweets = data_copy['text']
#data_copy.dropna(axis = 1, how ='all', inplace = True) 
data_copy = data_copy.dropna(axis=0, subset=['Tweet'])
count = data_copy['Tweet'].str.split().str.len()
data_copy = data_copy[~(count==0)]
  #data[~(count==1)]
#data_copy = data_copy[~data_copy['Tweet'].isnull()]

In [ ]:
len(data_copy)

In [ ]:
data_copy.head(20)

In [ ]:
from googletrans import Translator 
translator = Translator() 


In [ ]:
len(data_copy)

In [ ]:
data3 = data_copy.head(10)

In [ ]:
cities = data_copy['City']

In [ ]:
cities

In [ ]:
cities.to_csv("Cities.csv")

In [ ]:
translator = GoogleTranslate("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") #Insert your Google translate  key
    #translator = Translator()
translated_tweets = [translator.translate(tweet, "en") for tweet in data_copy['Tweet']]

translated = pd.DataFrame(translated_tweets)

# new_df = pd.concat([pd.DataFrame(translated_tweets),
#                        (data3['City'] * len(translated_tweets))],
#                        axis=1)
# if dataframe.empty:
#     dataframe = new_df
# else:
#        dataframe = pd.concat([dataframe, new_df], ignore_index=True)

In [ ]:
data2.to_csv("data1.csv")

In [ ]:
translated.to_csv("translated_tweets.csv")

In [ ]:
# data2['English'] = data2['Tweet'].apply(translator.translate,dest='en').apply(getattr, args=('text',))

In [ ]:
translated.head()

In [ ]:
data_cities = pd.read_csv("translated_cities.csv")

In [ ]:
data_cities.head()

In [ ]:
df = pd.read_csv('training.csv', encoding='latin-1')

# Drop unnecessary columns, leaving behind the [label, text] columns
df = df.drop(df.columns[[1, 2, 3, 4]], axis=1)

# Rename these columns
df.columns = ['label', 'text']

# Reverse the format to [text, label]
df = df[['text', 'label']]

In [ ]:
min(df.label)

In [ ]:
df.head()

In [ ]:
num_of_rows = df.shape[0]

In [ ]:
int(num_of_rows)

In [ ]:
# Shuffle the rows
msk = np.random.rand(len(df)) <= 0.7
train = df[msk]
valid = df[~msk]

In [ ]:
len(train)

In [ ]:
# Save back to .csv format
train.to_csv('train.csv', index=False)
valid.to_csv('valid.csv', index=False)

### Training  BERT 

In [ ]:
import torch
#import apex
import os
from pytorch_pretrained_bert.tokenization import BertTokenizer
from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy

In [ ]:
BERT_MODEL_PATH = './Bert_model/'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None)

In [ ]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

In [ ]:
os.listdir(BERT_MODEL_PATH)

In [ ]:
MODEL_PATH='./models/' 

In [ ]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
MODEL_PATH + 'pytorch_model.bin')

In [ ]:
bert_model = 'bert-base-uncased'

In [ ]:
bert_model

In [ ]:
DATA_PATH = './data/'     # path for data files (train and val)
LABEL_PATH = './labels/'  # path for labels file
MODEL_PATH='./models/'   # path for model artifacts to be stored
LOG_PATH='./logs/'      # path for log files to be stored



# Default args. If GPU runs out of memory while training, decrease training
# batch size

#tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, 
                                          #do_lower_case=args['do_lower_case'])
args = {
    
    "max_seq_length": 512,
    "do_lower_case": True,
    "train_batch_size": 8,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

# The tokenizer object is used to split the text into tokens used in training
tokenizer = BertTokenizer.from_pretrained(bert_model,
                                         do_lower_case=args['do_lower_case'])
    
# Check for GPU                                     
device = torch.device('cuda')
# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

# BertDataBunch contains the training, validation, and tests sets, alongside
# arguments and the tokenizer used in training
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer, 
                          train_file='train.csv', val_file='valid.csv', label_file='labels.csv',
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=False)


In [ ]:
BERT_PRETRAINED_PATH = "./Bert_model"

In [ ]:
# Choose the metrics used for the error function in training
metrics = []
metrics.append({'name': 'accuracy', 'function': accuracy})

# The learner contains the logic for training loop, validation loop, 
# optimiser strategies and key metrics calculation
learner = BertLearner.from_pretrained_model(
      databunch, BERT_PRETRAINED_PATH, metrics, device, logger, 
      finetuned_wgts_path=None, is_fp16=args['fp16'], 
      loss_scale=args['loss_scale'], multi_gpu=multi_gpu, multi_label=False)
      
# Train the model
learner.fit(6, lr=args['learning_rate'], 
            schedule_type="warmup_cosine_hard_restarts")
            
# Save the model into a file
learner.save_and_reload(MODEL_PATH, "model_sentiment")

## Predictions 

In [ ]:
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor(model_path=MODEL_PATH, pretrained_path=BERT_PRETRAINED_PATH, 
                                        label_path=LABEL_PATH, multi_label=False)
